In [ ]:
import os
import torch
import timm
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm


In [ ]:
# ImageNet stats
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load dataset
dataset = ImageFolder(root=r'C:\Users\shazi\OneDrive\Desktop\VS Code\fyp\skintypepatches 128x128', transform=train_transform)
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Replace transform for validation
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class_names = dataset.classes


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=len(class_names))
model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct = 0, 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)

    return total_loss / len(loader), correct.double() / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct = 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    return total_loss / len(loader), correct.double() / len(loader.dataset), y_true, y_pred


In [ ]:
val_loss, val_acc, y_true, y_pred = evaluate(model, val_loader, criterion)
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
torch.save(model.state_dict(), "vit_model_cip.pth")
#add the weight saving path here #

In [ ]:
import os
import numpy as np
import torch
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from tqdm import tqdm

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Dataset path
DATA_DIR = "Preprocessing/skintypepatches 224x224"

# ImageNet mean and std
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

# Strong data augmentation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(300, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(25),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

val_test_transforms = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

# Dataset loading
full_dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
num_classes = len(full_dataset.classes)

# Dataset split
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_ds, val_ds, test_ds = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Apply validation/test transforms
val_ds.dataset.transform = val_test_transforms
test_ds.dataset.transform = val_test_transforms

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

# EfficientNetB3 model
model = models.efficientnet_b3(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

# Class weights for imbalance handling
y_train = [label for _, label in train_ds]
class_weights = compute_class_weight(class_weight='balanced', classes=np.arange(num_classes), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Early stopping setup
best_acc = 0
patience, max_patience = 0, 5

# Training loop
def train_model(model, train_loader, val_loader, epochs=25):
    global best_acc, patience
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}: Val Accuracy = {acc:.4f}, Loss = {total_loss:.4f}")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_effnet_model.pth")
            print("✅ Model saved")
            patience = 0
        else:
            patience += 1
            if patience >= max_patience:
                print("🛑 Early stopping triggered")
                break
    return best_acc

best_val_acc = train_model(model, train_loader, val_loader)

# Load best model
model.load_state_dict(torch.load("best_effnet_model.pth"))
model.eval()

# Evaluation function
def evaluate(model, dataloader, true_labels):
    y_pred, y_probs = [], []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(probs, 1)
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    acc = accuracy_score(true_labels, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, y_pred, average='macro')
    auc = roc_auc_score(label_binarize(true_labels, classes=range(num_classes)), y_probs, multi_class='ovr')
    return acc, precision, recall, f1, auc

# Get labels
y_val = [label for _, label in val_ds]
y_test = [label for _, label in test_ds]

# Final evaluations
val_metrics = evaluate(model, val_loader, y_val)
test_metrics = evaluate(model, test_loader, y_test)

# Save metrics
metrics_df = pd.DataFrame({
    "Split": ["Validation", "Test"],
    "Accuracy": [val_metrics[0], test_metrics[0]],
    "Precision": [val_metrics[1], test_metrics[1]],
    "Recall": [val_metrics[2], test_metrics[2]],
    "F1 Score": [val_metrics[3], test_metrics[3]],
    "AUC-ROC": [val_metrics[4], test_metrics[4]],
})

metrics_df.to_csv("efficientnetb3_skin_metrics.csv", index=False)
print("📈 Metrics saved to efficientnetb3_skin_metrics.csv")
print(metrics_df)

In [ ]:
import os
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
import timm
from tqdm import tqdm

# Directory paths
SOURCE_TRAIN_DIR = "Preprocessing/skintypepatches 128x128"
SOURCE_TEST_DIR = "Preprocessing/skintypepatches 128x128"
COMBINED_DIR = "/kaggle/working/combined"
TEST_DIR = "/kaggle/working/test"
TRAIN_DIR = "/kaggle/working/train"
VAL_DIR = "/kaggle/working/val"
# Create directories if they don't exist
for dir_path in [COMBINED_DIR, TEST_DIR, TRAIN_DIR, VAL_DIR]:
    os.makedirs(dir_path, exist_ok=True)

def is_valid_image(file_path):
    try:
        img = Image.open(file_path)
        img.verify()
        return True
    except:
        return False

def copy_images(source_dir, dest_dir):
    for subfolder in os.listdir(source_dir):
        full_path = os.path.join(source_dir, subfolder)
        for img in os.listdir(full_path):
            img_path = os.path.join(full_path, img)
            if is_valid_image(img_path):
                dest_folder = os.path.join(dest_dir, subfolder)
                os.makedirs(dest_folder, exist_ok=True)
                shutil.copy(img_path, os.path.join(dest_folder, img))

# Copy images to combined directory
for source_dir in [SOURCE_TRAIN_DIR, SOURCE_TEST_DIR]:
    copy_images(source_dir, COMBINED_DIR)

def split_data(combined_dir, test_dir, train_dir, val_dir, test_size=20, val_split=0.2):
    all_combined_images = []
    for subfolder in os.listdir(combined_dir):
        subfolder_path = os.path.join(combined_dir, subfolder)
        all_combined_images += [os.path.join(subfolder_path, img) for img in os.listdir(subfolder_path)]

    random.shuffle(all_combined_images)

    test_images = random.sample(all_combined_images, test_size)
    for img_path in test_images:
        dest_folder = os.path.join(test_dir, os.path.basename(os.path.dirname(img_path)))
        os.makedirs(dest_folder, exist_ok=True)
        shutil.move(img_path, os.path.join(dest_folder, os.path.basename(img_path)))
        all_combined_images.remove(img_path)

    split_index = int((1 - val_split) * len(all_combined_images))
    train_images = all_combined_images[:split_index]
    val_images = all_combined_images[split_index:]

    for image_list, folder in [(train_images, train_dir), (val_images, val_dir)]:
        for img_path in image_list:
            dest_folder = os.path.join(folder, os.path.basename(os.path.dirname(img_path)))
            os.makedirs(dest_folder, exist_ok=True)
            shutil.copy(img_path, os.path.join(dest_folder, os.path.basename(img_path)))

# Split data into train, validation, and test sets
split_data(COMBINED_DIR, TEST_DIR, TRAIN_DIR, VAL_DIR)

def load_and_visualize_data(data_directory, num_images_per_subfolder=4):
    class_labels = os.listdir(data_directory)
    fig, axes = plt.subplots(len(class_labels), num_images_per_subfolder, figsize=(10, 10))
    for i, class_label in enumerate(class_labels):
        class_dir = os.path.join(data_directory, class_label)
        image_paths = [os.path.join(class_dir, img) for img in os.listdir(class_dir)[:num_images_per_subfolder]]
        images = [plt.imread(img_path) for img_path in image_paths]
        for j, ax in enumerate(axes[i]):
            img = images[j]
            ax.imshow(img)
            ax.set_title(class_label)
            ax.axis("off")
    plt.tight_layout()
    plt.show()

# Visualize training data
load_and_visualize_data(TRAIN_DIR)

# Data augmentation and transformation
augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomAffine(degrees=0, shear=10),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets and dataloaders
training_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=augmentations)
validation_dataset = datasets.ImageFolder(root=VAL_DIR, transform=transform)

train_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Model definition
model = timm.create_model('efficientnet_b3', pretrained=True)
num_classes = 4
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Training and validation loop
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10):
    best_acc = 0
    for epoch in range(epochs):
        model.train()
        train_correct, train_total, total_loss = 0, 0, 0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).sum().item()
            train_total += labels.size(0)
        print(f"[{epoch+1}/{epochs}] Train Loss: {total_loss/len(train_loader):.4f} | Accuracy: {train_correct/train_total:.4f}")

        model.eval()
        val_correct, val_total, val_loss = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_total += labels.size(0)
        acc = val_correct / val_total
        print(f"Validation Loss: {val_loss/len(val_loader):.4f} | Accuracy: {acc:.4f}")
        scheduler.step(val_loss)

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "efficientnet_b3_best.pth")

# Train and validate the model
train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, EPOCHS)

# Load the best model and evaluate
model.load_state_dict(torch.load("efficientnet_b3_best.pth"))
model.eval()

def evaluate_model(model, val_loader):
    all_predictions, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            all_predictions.extend(torch.argmax(outputs, 1).cpu().numpy())
            all_labels.extend(labels.numpy())
    return all_predictions, all_labels

all_predictions, all_labels = evaluate_model(model, val_loader)

# Calculate metrics
acc = accuracy_score(all_labels, all_predictions)
prec = precision_score(all_labels, all_predictions, average='weighted')
rec = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1-score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
df_cm = pd.DataFrame(cm, index=training_dataset.classes, columns=training_dataset.classes)
plt.figure(figsize=(8, 6))
sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

# Test data transformation and prediction
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class_labels = ['oily','normal','dry']

def predict_and_visualize_test_images(test_dir, model, class_labels, num_images_per_class=3):
    images = []
    for class_name in os.listdir(test_dir):
        class_path = os.path.join(test_dir, class_name)
        for img_name in os.listdir(class_path)[:num_images_per_class]:
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path).convert('RGB')
            img_tensor = test_transform(img).unsqueeze(0).to(device)
            with torch.no_grad():
                pred = model(img_tensor).argmax(1).item()
            label = class_labels[pred]
            correct = "Correct" if label == class_name else "Wrong"
            images.append((img, label, correct))
    return images

images = predict_and_visualize_test_images(TEST_DIR, model, class_labels)

# Visualize test predictions
rows = int(np.ceil(len(images) / 3))
fig, axs = plt.subplots(rows, 3, figsize=(15, 5 * rows))
for i in range(rows):
    for j in range(3):
        idx = i * 3 + j
        if idx < len(images):
            img, label, result = images[idx]
            axs[i, j].imshow(img)
            axs[i, j].set_title(f"{label} ({result})")
            axs[i, j].axis("off")
plt.tight_layout()
plt.show()

Batch 8 all param

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb  
import clip
from torch.optim import AdamW
from torch.optim import SGD
from pytorch_lamb import Lamb
from torch_optimizer import RAdam


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")



batch_sizes = [8]
learning_rates = [0.1,0.01,0.001,0.0001,0.00001]
optimizers_list = ['LAMB', 'AdamW', 'SGD', 'RAdam']
num_classes=3
total_epochs = 501
start=100
step=5



clip_model, _ = clip.load("ViT-B/32", device=device)



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])



data_dir = "Preprocessing/stage2patches"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)



train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


for param in clip_model.visual.parameters():
    param.requires_grad = False


class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits






def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return AdamW(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'LAMB':
        return Lamb(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        return SGD(model_params, lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == 'RAdam':
        return RAdam(model_params, lr=lr, weight_decay=weight_decay)


def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse



def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse



def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse




for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            model = CLIPSkinClassifier(clip_model, num_classes=num_classes).to(device).float()
            
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
            
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            
            
            criterion = nn.CrossEntropyLoss()
            optimizer = get_optimizer(optimizer_name,model.parameters(), lr=lr)
            
            
            for epoch in range(start,total_epochs,step):
                
                
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)


                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name}, \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc if train_roc_auc is not None else 'N/A'}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name}, \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc if train_roc_auc is not None else 'N/A'}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc if train_roc_auc is not None else 'N/A'}, Test RMSE: {test_rmse:.4f}")

                
                
                overall_result = {
                    'Epoch': epoch ,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'Batch 8 all parameter testing.csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)